<a href="https://colab.research.google.com/github/nbeaudoin/LangChain-Experimentation-Zone/blob/main/LangChain_Chat_with_Flan20B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial Source: https://www.youtube.com/watch?v=VW5LBavIfY4&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=5

Finetuned LAnguage Net (FLAN)

In [3]:
!pip -q install openai python-dotenv langchain transformers huggingface
import os
import openai
from dotenv import load_dotenv

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load variables from .env file
load_dotenv('/content/drive/MyDrive/Projects/keys/secrets.json')

# Use variables
huggingface_api = os.getenv('HUGGINGFACE_API_KEY')

In [ ]:
from langchain.llms import HuggingFaceHub

In [ ]:
flan_ul2 = HuggingFaceHub(
    repo_id="google/flan-ul2",
    huggingfacehub_api_token=huggingface_api,
    model_kwargs={"temperature":0.1,
                  "max_new_tokens":256}
)

In [ ]:
flan_T5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    huggingfacehub_api_token=huggingface_api,
    model_kwargs={"temperature":0}
)

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
conversation = ConversationChain(
    llm=flan_ul2,
    verbose=True,
    memory=memory
)

In [ ]:
conversation.predict(input="Hi there! I am Sam")

In [ ]:
conversation.predict(input="How are you today?")

## Counting the tokens

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

In [ ]:
test_input = "Now Good Morning Ms Rogers"
tokenizer([test_input])
tokenizer.tokenize(test_input)

In [ ]:
memory.buffer

In [ ]:
formatted_prompt = conversation.prompt.format(input='the next input', history=memory.buffer)
formatted_prompt

In [ ]:
def chat_to_llm(chat_llm):
  conversation_total_tokens = 0
  new_conversation = ConversationChain(llm=chat_llm,
                                       verbose=False,
                                       memory=ConversationBufferMemory())

  while True:
    message = input("Human: ")
    if message == 'exit':
      print(f"{conversation_total_tokens} tokens used in total in this conversation")
      break
    if message:
      formatted_prompt = conversation.prompt.format(input=message, history=memory.buffer)
      num_tokens = len(tokenizer.tokenize(formatted_prompt))
      conversation_total_tokens += num_tokens
      print(f"tokens sent {num_tokens}")
      response = new_conversation.predict(input=message)
      response_num_tokens = len(tokenizer.tokenize(response))
      conversation_total_tokens += response_num_tokens
      print(f"LLM: {response}")

In [ ]:
chat_to_llm(flan_ul2)

In [ ]:
chat_to_llm(flan_T5)